# Scraping Top Bestseller Book in Amazon Project Using Python

**Amazon Bestseller Books** website is a popular website where people can find top popular books. This site contains many genres. In each genre, books are ranked based on rating and the number of buyers. Some genres includes action & adventure, travel, and romance. In general, information of a book contains name, author, rating, number of reviewers, and its URL. They are packed in a tag, and our job is to use function to extract the information from the tag.
![](https://hips.hearstapps.com/hmg-prod.s3.amazonaws.com/images/best-selling-books-2020-1607638264.png)

**Purpose**: This project showed the method to scrape the information of the bestseller books such as name, author, rating, and URL in each genre in Amazone website: https://www.amazon.in/gp/bestsellers/books/. The information was represented as data frame, and was saved in csv file for each category. To organize easily, a folder was created to contained all the csv files at the final.
- During introducing procedure, this project also mentioned tools used such as Python, requests, Beautiful Soup, Pandas..., which were used for scraping the bestseller books in Amazon website
![](https://imgur.com/adDRt5p)

**Project Outline:**
- Scraped Website: https://www.amazon.in/gp/bestsellers/books/
- Get a list of genres. For each genre, we'll get the genre name and genre page URL
- For each genre, we'll get information of the bestseller books from that genre page.
- For each book, we'll get name of the book, author name, rating, and the book's URL.
- For example, in action & adventure genre:
```
Book Name,Author,Rating,URL
Harry Potter and the Philosopher's Stone,J.K. Rowling,4.7 out of 5 stars,https://www.amazon.in/Harry-Potter-Philosophers-Stone-Rowling-ebook/dp/B019PIOJYU/ref=zg_bs_g_1318158031_d_sccl_1/000-0000000-0000000?psc=1
THE LION INSIDE,Rachel Bright,4.7 out of 5 stars,https://www.amazon.in/LION-INSIDE-Rachel-Bright/dp/1408349043/ref=zg_bs_g_1318158031_d_sccl_2/000-0000000-0000000?psc=1
"THE SILENT PATIENT [Paperback] Michaelides, Alex",Alex Michaelides,4.5 out of 5 stars,https://www.amazon.in/Silent-Patient-Alex-Michaelides/dp/1409181634/ref=zg_bs_g_1318158031_d_sccl_3/000-0000000-0000000?psc=1
The Housemaid : An addictive psychological thriller with mind-bending twists,Freida McFadden,4.4 out of 5 stars,https://www.amazon.in/Housemaid-addictive-psychological-thriller-mind-bending/dp/014346115X/ref=zg_bs_g_1318158031_d_sccl_4/000-0000000-0000000?psc=1
The Hidden Hindu Boxset: The Complete Trilogy | Bestselling Sci-fi Novels by Akshat Gupta,Akshat Gupta,4.5 out of 5 stars,https://www.amazon.in/Hidden-Hindu-Boxset-Akshat-Gupta/dp/0143467352/ref=zg_bs_g_1318158031_d_sccl_5/000-0000000-0000000?psc=1
The Stationery Shop of Tehran,Marjan Kamali,4.4 out of 5 stars,https://www.amazon.in/Stationery-Shop-Tehran-Marjan-Kamali/dp/147118501X/ref=zg_bs_g_1318158031_d_sccl_6/000-0000000-0000000?psc=1
Harry Potter and the Chamber of Secrets,J.K. Rowling,4.7 out of 5 stars,https://www.amazon.in/Harry-Potter-Chamber-Secrets-Rowling-ebook/dp/B019PIOJY0/ref=zg_bs_g_1318158031_d_sccl_7/000-0000000-0000000?psc=1
```

## 1. Scrape the category page from Amazon

**Objective:** To obtain the list of book category from the bestseller website and their urls.

- **Procedure**:
    - Use requests to download the Amazone Bestseller page.
    - Use BS4 to parse and extract information from each category page (name and url of each category)
        - To extract category name and its url, two functions were created: get_category_name(doc) and get_category_url(doc).
    - Convert to a pandas dataframe containing name and URL of category page.

**Install and Import necessary libraries:**

In [77]:
import requests
import pandas as pd
import os
from bs4 import BeautifulSoup

**Function to download the Bestseller Amazon page:**

In [78]:
def get_amazon_page():
    #TODO: this function return the book category page
    url = 'https://www.amazon.in/gp/bestsellers/books/'
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception('Failed to load page {}'.format(url))
    doc = BeautifulSoup(response.text,'html.parser')
    return doc

In [79]:
doc = get_amazon_page()

In [80]:
type(doc)

bs4.BeautifulSoup

**Some helper functions to parse and extract information of each book category page:**

In [81]:
def get_category_name(cat_doc):
    # Result the name of each category
    select_class="_p13n-zg-nav-tree-all_style_zg-browse-item__1rdKf _p13n-zg-nav-tree-all_style_zg-browse-height-large__1z5B8";
    class_tags = cat_doc.find_all(class_=select_class)
    categories_tag = class_tags[1:]
    name_categories = [];
    for tag in categories_tag:
        name_categories.append(tag.text)
    return name_categories

 `get_category_name()` received the page content. Then it find all tags containing name of the book category, extract, and return the name of each book category.

In [82]:
def get_category_url(cat_doc):
    # Return the url of each category
    select_class="_p13n-zg-nav-tree-all_style_zg-browse-item__1rdKf _p13n-zg-nav-tree-all_style_zg-browse-height-large__1z5B8";
    class_tags = cat_doc.find_all(class_=select_class)
    categories_tag = class_tags[1:]
    category_urls = [];
    base_url = 'https://www.amazon.in';
    for tag in categories_tag:
        a_tag = tag.find('a')
        category_urls.append(base_url + a_tag['href'])
    return category_urls

 `get_category_url()` finds all tags of categories and extract, and return URL information.

- **Explain:**
    - From the bestseller page, first inspect each category URL, and find the tag represent for each category page.
    - to find the name and link of each category, use `find_all` to find all tags that has the `class`: `_p13n-zg-nav-tree-all_style_zg-browse-item__1rdKf _p13n-zg-nav-tree-all_style_zg-browse-height-large__1z5B8`
    - to get the name, simply use `.text` for each tag above.
    - to get the link, we need to create a loop, for each tag, use `find` to search `a` tag with attribute `href`. Finally, combine with the `base_url: https://www.amazon.in` to form the complete link
   
![](https://i.imgur.com/URwK9Dy.png)

**Results:**

In [83]:
book_categories = get_category_name(doc)

In [84]:
len(book_categories)

35

In [85]:
book_categories[:5]

['Action & Adventure',
 'Arts, Film & Photography',
 'Biographies, Diaries & True Accounts',
 'Business & Economics',
 "Children's Books"]

In [86]:
category_urls = get_category_url(doc);
len(category_urls)

35

In [87]:
category_urls[:5]

['https://www.amazon.in/gp/bestsellers/books/1318158031',
 'https://www.amazon.in/gp/bestsellers/books/1318052031',
 'https://www.amazon.in/gp/bestsellers/books/1318064031',
 'https://www.amazon.in/gp/bestsellers/books/1318068031',
 'https://www.amazon.in/gp/bestsellers/books/64619755031']

## 2. Get info of bestseller books in the page of each genre

**Objective**: In this section, we will parse and extract info of each bestseller book for each genre such as book name, author, rating, and URL for EACH category page.

- **Procedure**:
    - Use requests to download a category page.
    - Use functions to find all tags containing information such as name, author, rating, and URL of books and pass the tag one-by-one to helper functions.
    - Use helper functions that take a tag of each book and return the info such as name, author, rate, and URL separately
    - Convert to a pandas dataframe containing information of books.

**For example:** Let take the first genre: Action and Adventure. Below is the URL of action and adventure category.

**Function to download a category page:**

In [88]:
def get_category_page(cat_url):
    # This function download a category page
    response = requests.get(cat_url)
    if response.status_code != 200:
        raise Exception('Failed to load page {}'.format(cat_url))
    cat_doc = BeautifulSoup(response.text,'html.parser')
    return cat_doc

 `get_category_page()` is used to download each book category page.

In [103]:
cat_doc = get_category_page(category_urls[0])

In [104]:
type(cat_doc)

bs4.BeautifulSoup

In [105]:
book_class = 'zg-grid-general-faceout';
book_tags = cat_doc.find_all(class_=book_class);
tag = book_tags[10];

In [106]:
#bookname_class = "_cDEzb_p13n-sc-css-line-clamp";
name_tag = tag.select('div[class*="_cDEzb_p13n-sc-css-line-clamp"]');
len(name_tag)

2

**Function to get info of books:**

In [93]:
def get_category_book(book_doc):
    #Describe: this function return the dataframe of books info
    
    #Extract book name and author tags
    book_class = 'zg-grid-general-faceout';
    book_tags = book_doc.find_all(class_=book_class);
    
    #Get book info
    book_category_dict = {
    'Book Name' :[],
    'Author' :[],
    'Rating' :[],
    'URL' :[],
    }
    
    for i in range(len(book_tags)):
        book_info = get_book_category_info(book_tags[i]);
        book_category_dict['Book Name'].append(book_info[0]);
        book_category_dict['Author'].append(book_info[1]);
        book_category_dict['Rating'].append(book_info[2]);
        book_category_dict['URL'].append(book_info[3]);
    return pd.DataFrame(book_category_dict)

 `get_category_book()` is used to find all tags for each book by specifying the class attribute. These tags will be passed one-by-one to another helper function `get_book_category_info`, which return the lists of book name, author, rating and URL. This function also creates the dataframe containing info of each book such as name, author, rating and URL

**Help Function taking each tag and return info of each book:**

In [94]:
def get_book_category_info(book_tag):
    # Describe: This function takes each tag and return info for each book
    #Book & Author name
    name, author = book_author_name(book_tag);
    #link book
    link = url(book_tag);
    #Rating: contain check if there is rating for a book.
    book_rate = rating(book_tag);
    return name,author,book_rate,link

 `get_book_category_info()` recieves the tag of each book. Then it has 4 helper inner functions to extrac info of book such as name, author, rating and URL.
 - `book_name` used to return the name of the book
 - `book_author` used to return the name of author
 - `rating` used to return the rating of the book
 - `link` used to return the URL of the book

In [107]:
def book_author_name(tag):
    # This function return book name fromt tag
    name_tag = tag.select('div[class*="_cDEzb_p13n-sc-css-line-clamp"]');
    if len(name_tag)<=1:
        name= name_tag[0].text;
        author="";
        return name, author
    return name_tag[0].text,name_tag[1].text;
    
def rating(tag):
    # This function return the rating. It also ontain check if there is rating for a book.
    rate_tag = tag.find(class_='a-icon-alt')
    if rate_tag == None:
        book_rate = 'NA'
    else:
        book_rate = rate_tag.text
    return book_rate

def url(tag):
    base_url = 'https://www.amazon.in'
    a_tag = tag.find('a')
    return base_url + a_tag['href']

- **Explain Procedure:**
    - First, for each category, download the category page by `get_category_page` by passing its URL to bs4.
    - Using `get_category_book()` to find list of tags containing info of all books by using `find_all` to find all tags that has the `class`: `zg-grid-general-faceout`.
    - Then create a loop to pass every tag of each book to the helper function `get_book_category_info()`. This function will return the name, author, rating and book url.
    - In `get_book_category_info()`, we have 4 inner helper functions that extract 4 types of info: `book_name`, `book_author`, `rating`, and `url`.
   
![](https://i.imgur.com/rJG6TPJ.png)

**Example Result:**

In [108]:
get_category_book(cat_doc)[:5]

,Book Name,Author,Rating,URL
0,Harry Potter and the Philosopher's Stone,J.K. Rowling,4.7 out of 5 stars,https://www.amazon.in/Harry-Potter-Philosopher...
1,THE LION INSIDE,Rachel Bright,4.7 out of 5 stars,https://www.amazon.in/LION-INSIDE-Rachel-Brigh...
2,"THE SILENT PATIENT [Paperback] Michaelides, Alex",Alex Michaelides,4.5 out of 5 stars,https://www.amazon.in/Silent-Patient-Alex-Mich...
3,The Housemaid : An addictive psychological thr...,Freida McFadden,4.4 out of 5 stars,https://www.amazon.in/Housemaid-addictive-psyc...
4,The Hidden Hindu Boxset: The Complete Trilogy ...,Akshat Gupta,4.5 out of 5 stars,https://www.amazon.in/Hidden-Hindu-Boxset-Aksh...


## 3. Saving and Organizing data

**Function to scrape info of bestseller books in ALL catgories and save as csv files in each category folder.**

In [109]:
def scrape_book_bestseller():
    print('Scraping the top bestseller book in AMAZON:')
    category_df = scrape_category()
    
    os.makedirs('data',exist_ok=True)
    for index,row in category_df.iterrows():
        print('Scraping the top bestsellers book for {} genre'.format(row['Book Categories']))
        scrape_category_book(row['URL'],'data/{}.csv'.format(row['Book Categories']))

- The function above is the general outermost function, and is the function we'll call to scrape all info of books in ALL categories. It has two helper functions:
    - `scrape_category()` to scrape the name of each category and the URL and create a data frame. It contains `get_amazon_page` helper function to download bestseller book page by using requests. Moreover, this function contain 2 inner helper functions `get_category_name` and `get_category_url` above to scrape the info of each category.

In [110]:
def scrape_category():
    doc =  get_amazon_page();
    category_dict ={
    'Book Categories': get_category_name(doc),
    'URL': get_category_url(doc)
    }
    return pd.DataFrame(category_dict);

- `scrape_category_book()` to scrape books in each category. It belongs to a loop that pass name and URL of each category in the list above. Inside this function, we see the `get_category_book` and `get_category_page` function.
    - First `scrape_category_book` recieve the URL of a category and its saving path. Then the function check if the saving path exists. If so, the cvs file is created, and no need for scraping. If not, it will pass the url to `get_category_page()` function to download the category page by using bs4. Then `get_category_book()` take file doc from the previous function to parse and extract info of each book in the category page and create the list. As we know, inside `get_category_book()`, we have another helper function that take a tag of each book to extract the information.
    - This function also has a checkpoint. During running the program, there will be error with fetching the web page. So, to avoid create the same csv file. It wil check the if the csv file exists by checking the savving path. If the file exists, that category will be skipped.

In [111]:
def scrape_category_book(cat_url,path):
    if os.path.exists(path):
        print('The file {} already exists. Skipping...'.format(path))
        return
    book_df = get_category_book(get_category_page(cat_url))
    book_df.to_csv(path, index = None)

## 4. Put it together - Final Code

#### a. Function of scraping category

In [112]:
def get_category_name(cat_doc):
    # Result the name of each category
    select_class="_p13n-zg-nav-tree-all_style_zg-browse-item__1rdKf _p13n-zg-nav-tree-all_style_zg-browse-height-large__1z5B8";
    class_tags = cat_doc.find_all(class_=select_class)
    categories_tag = class_tags[1:]
    name_categories = [];
    for tag in categories_tag:
        name_categories.append(tag.text)
    return name_categories

def get_category_url(cat_doc):
    # Return the url of each category
    select_class="_p13n-zg-nav-tree-all_style_zg-browse-item__1rdKf _p13n-zg-nav-tree-all_style_zg-browse-height-large__1z5B8";
    class_tags = cat_doc.find_all(class_=select_class)
    categories_tag = class_tags[1:]
    category_urls = [];
    base_url = 'https://www.amazon.in';
    for tag in categories_tag:
        a_tag = tag.find('a')
        category_urls.append(base_url + a_tag['href'])
    return category_urls

def get_amazon_page():
    #TODO: this function return the book category page
    url = 'https://www.amazon.in/gp/bestsellers/books/'
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception('Failed to load page {}'.format(url))
    doc = BeautifulSoup(response.text,'html.parser')
    return doc

def scrape_category():
    # This function return the list of category contain name and URL
    doc =  get_amazon_page();
    category_dict ={
    'Book Categories': get_category_name(doc),
    'URL': get_category_url(doc)
    }
    return pd.DataFrame(category_dict);

#### b. Function of scraping info of each books for each category page.

In [113]:
def book_author_name(tag):
    # This function return book name fromt tag
    name_tag = tag.select('div[class*="_cDEzb_p13n-sc-css-line-clamp"]');
    if len(name_tag)<=1:
        name= name_tag[0].text;
        author="";
        return name, author
    return name_tag[0].text,name_tag[1].text;

def rating(tag):
    # This function return the rating. It also ontain check if there is rating for a book.
    rate_tag = tag.find(class_='a-icon-alt')
    if rate_tag == None:
        book_rate = 'NA'
    else:
        book_rate = rate_tag.text
    return book_rate

def url(tag):
    base_url = 'https://www.amazon.in'
    a_tag = tag.find('a')
    return base_url + a_tag['href']
    
def get_book_category_info(book_tag):
    # Describe: This function takes each tag and return info for each book
    #Book & Author name
    name, author = book_author_name(book_tag);
    #link book
    link = url(book_tag);
    #Rating: contain check if there is rating for a book.
    book_rate = rating(book_tag);
    return name,author,book_rate,link

def get_category_book(book_doc):
    #Describe: this function return the dataframe of books info
    #Extract book name and author tags
    book_class = 'zg-grid-general-faceout';
    book_tags = book_doc.find_all(class_=book_class);
    
    #Get book info
    book_category_dict = {
    'Book Name' :[],
    'Author' :[],
    'Rating' :[],
    'URL' :[],
    }
    
    for i in range(len(book_tags)):
        book_info = get_book_category_info(book_tags[i]);
        book_category_dict['Book Name'].append(book_info[0]);
        book_category_dict['Author'].append(book_info[1]);
        book_category_dict['Rating'].append(book_info[2]);
        book_category_dict['URL'].append(book_info[3]);
    return pd.DataFrame(book_category_dict)

def scrape_category_book(cat_url,path):
    # this function accept the url of each category, scrape the book info and save it in specific path.
    if os.path.exists(path):
        print('The file {} already exists. Skipping...'.format(path))
        return
    book_df = get_category_book(get_category_page(cat_url))
    book_df.to_csv(path, index = None)

#### c. General/ Outer function of scraping the list of categores and the info of books for each category page.

In [114]:
def scrape_book_bestseller():
    print('Scraping the top bestseller book in AMAZON:')
    category_df = scrape_category()
    
    os.makedirs('data',exist_ok=True)
    for index,row in category_df.iterrows():
        print('Scraping the top bestsellers book for {} genre'.format(row['Book Categories']))
        scrape_category_book(row['URL'],'data/{}.csv'.format(row['Book Categories']))

## 5. Results

**Check the function that scrape the list of categories:**

In [115]:
scrape_category()

,Book Categories,URL
0,Action & Adventure,https://www.amazon.in/gp/bestsellers/books/131...
1,"Arts, Film & Photography",https://www.amazon.in/gp/bestsellers/books/131...
2,"Biographies, Diaries & True Accounts",https://www.amazon.in/gp/bestsellers/books/131...
3,Business & Economics,https://www.amazon.in/gp/bestsellers/books/131...
4,Children's Books,https://www.amazon.in/gp/bestsellers/books/646...
5,Comics & Mangas,https://www.amazon.in/gp/bestsellers/books/131...
6,"Computing, Internet & Digital Media",https://www.amazon.in/gp/bestsellers/books/131...
7,"Crafts, Home & Lifestyle",https://www.amazon.in/gp/bestsellers/books/131...
8,"Crime, Thriller & Mystery",https://www.amazon.in/gp/bestsellers/books/131...
9,Engineering,https://www.amazon.in/gp/bestsellers/books/229...


**Check the final/outer code that scrape the bestseller books for each category and save it in csv file.**

In [116]:
scrape_book_bestseller()

Scraping the top bestseller book in AMAZON:
Scraping the top bestsellers book for Action & Adventure genre
Scraping the top bestsellers book for Arts, Film & Photography genre
Scraping the top bestsellers book for Biographies, Diaries & True Accounts genre
Scraping the top bestsellers book for Business & Economics genre
Scraping the top bestsellers book for Children's Books genre
Scraping the top bestsellers book for Comics & Mangas genre
Scraping the top bestsellers book for Computing, Internet & Digital Media genre
Scraping the top bestsellers book for Crafts, Home & Lifestyle genre
Scraping the top bestsellers book for Crime, Thriller & Mystery genre
Scraping the top bestsellers book for Engineering genre
Scraping the top bestsellers book for Exam Preparation genre
Scraping the top bestsellers book for Fantasy, Horror & Science Fiction genre
Scraping the top bestsellers book for Health, Family & Personal Development genre
Scraping the top bestsellers book for Health, Fitness & Nutri

**Comment:** There are several errors with loading the website. When running again, the code will check if the csv file exits. If so, it will skipp that category and move to the next category. The code work well and seems to scrape all categories. Let check csv files for some categories.

**History genre:**

In [117]:
# Display top 10 bestseller books in history genre
history_df = pd.read_csv('./data/History.csv');
history_df[:10]

,Book Name,Author,Rating,URL
0,Animals Tales From Panchtantra: Timeless Stori...,Wonder House Books,4.5 out of 5 stars,https://www.amazon.in/Animals-Tales-Panchtantr...
1,Autobiography of a Yogi,Paramahansa Yogananda,4.6 out of 5 stars,https://www.amazon.in/Autobiography-Yogi-Param...
2,Strictly Personal: Gursharan and Manmohan,Daman Singh,4.5 out of 5 stars,https://www.amazon.in/Strictly-Personal-Gursha...
3,Meditations,Marcus Aurelius,4.3 out of 5 stars,https://www.amazon.in/Meditations-Marcus-Aurel...
4,The Golden Road : How Ancient India Transforme...,Wlliam Dalrymple,4.6 out of 5 stars,https://www.amazon.in/Golden-Road-Ancient-Indi...
5,Nexus: A Brief History of Information Networks...,Yuval Noah Harari,4.5 out of 5 stars,https://www.amazon.in/Nexus-Brief-History-Info...
6,Jaun Elia: Ek Ajab Ghazab Shayar - Hindi,Jaun Elia,4.5 out of 5 stars,https://www.amazon.in/Jaun-Elia-Ajab-Ghazab-Sh...
7,Thirteen Months In The Himalayas : Chronicles ...,Om Swami,4.9 out of 5 stars,https://www.amazon.in/Thirteen-Months-Himalaya...
8,Nexus: FROM THE MULTI-MILLION COPY BESTSELLING...,Yuval Noah Harari,4.5 out of 5 stars,https://www.amazon.in/Nexus-MULTI-MILLION-BEST...
9,Man's Search For Meaning: The classic tribute ...,Victor E. Frankl,4.5 out of 5 stars,https://www.amazon.in/Mans-Search-Meaning-Vikt...


**Romance genre:**

In [118]:
# Display top 10 bestseller books in romance genre
romance_df = pd.read_csv('./data/Romance.csv');
romance_df[:10]

,Book Name,Author,Rating,URL
0,உயிரழகே என்னருகே (Tamil Edition),சர ணிகா,4.2 out of 5 stars,https://www.amazon.in/%E0%AE%89%E0%AE%AF%E0%AE...
1,காதல் என்னை காதலிக்கவில்லை : Kaathal Ennai Kaa...,Infaa Alocious L,4.4 out of 5 stars,https://www.amazon.in/%E0%AE%95%E0%AE%BE%E0%AE...
2,"Too Good to Be True : A smart, funny will-they...",Prajakta Koli,NaN,https://www.amazon.in/Too-Good-Be-True-will-th...
3,மறக்குமா மாமன் உள்ளம்? (Tamil Edition),தேன் நிலா J,4.5 out of 5 stars,https://www.amazon.in/%E0%AE%AE%E0%AE%B1%E0%AE...
4,எனை ஆட்கொள்வாயா உன் இதயச் சிறையில்! (Tamil Edi...,ஜீஷா ஶ்ரீ Jeesha sri,5.0 out of 5 stars,https://www.amazon.in/%E0%AE%8E%E0%AE%A9%E0%AF...
5,விழி அழகில் மீண்டேனடி (Tamil Edition),மீனா கண்ணன் Meena kannan,4.5 out of 5 stars,https://www.amazon.in/%E0%AE%85%E0%AE%B4%E0%AE...
6,அதிரடிக்காரன் மச்சான் டி [ Athiradikkaran Mach...,Pradhanya Kuzhali,4.3 out of 5 stars,https://www.amazon.in/%E0%AE%85%E0%AE%A4%E0%AE...
7,Cruel Winter with You (Under the Mistletoe col...,Ali Hazelwood,4.0 out of 5 stars,https://www.amazon.in/Cruel-Winter-Under-Mistl...
8,ஜனகரின் பொம்மி (Tamil Edition),SAKTHI GURU,4.5 out of 5 stars,https://www.amazon.in/%E0%AE%9C%E0%AE%A9%E0%AE...
9,மேக மழைச்சாரல் : Mega mazhai saral (Tamil Edit...,Meera Krish,4.5 out of 5 stars,https://www.amazon.in/%E0%AE%AE%E0%AF%87%E0%AE...


**Travel genre:**

In [119]:
# Display top 10 bestseller books in travel genre
travel_df = pd.read_csv('./data/Travel.csv');
travel_df[:10]

,Book Name,Author,Rating,URL
0,Days at the Morisaki Bookshop : The Internatio...,Satoshi Yagisawa,4.3 out of 5 stars,https://www.amazon.in/Days-Morisaki-Bookshop-I...
1,Children's First Mythology Stories - Pack of 5...,Wonder House Books,4.5 out of 5 stars,https://www.amazon.in/Childrens-First-Mytholog...
2,Relaxing Mandala Coloring Book For Adults,Wonder House Books,4.4 out of 5 stars,https://www.amazon.in/Relaxing-Mandala-Colorin...
3,Masala Lab,Ashok Krish,4.5 out of 5 stars,https://www.amazon.in/Masala-Lab-Science-India...
4,Normal People,Sally Rooney,4.1 out of 5 stars,https://www.amazon.in/Normal-People-Sally-Roon...
5,SATTVIK COOKING: MODERN AVATARS OF VEDIC FOODS,Hansaji J. Yogendra,4.4 out of 5 stars,https://www.amazon.in/SATTVIK-COOKING-MODERN-A...
6,You are the Best Friend: A True story of falli...,Ajay K Pandey,4.4 out of 5 stars,https://www.amazon.in/You-are-Best-Friend-Auth...
7,Flowers Coloring Book for adults,Wonder House Books,4.4 out of 5 stars,https://www.amazon.in/Flowers-Coloring-adults-...
8,Zen: The Art of Simple Living,Shunmyo Masuno,4.6 out of 5 stars,https://www.amazon.in/Zen-Simple-Living-Shunmy...
9,India & World Map ( Both Political & Physical ...,Vidya Chitr Prakashan,4.4 out of 5 stars,https://www.amazon.in/Political-Physical-Const...


## 7. Summary

Here is what we've covered in this notebook:
1. Download the bestseller book website of Amazon using `requests`.
2. Parse the HTML source code using beautiful soup.
3. Extract the name and URL of each category page in form of data frame.
4. For each book category:
    - Parse the HTML source code for each category page URL using beautiful soup.
    - Extract name, author, rating, and URL of each bestseller book.
    - Compile the information into a dictionary.
    - Return the list of books with informations: name, author...
5. For each category/ genre, we save the data frame into a separate csv file, and named with category.
6. During the running, there will be error with loading the website. To avoid create duplicate csv file, a checkpoint is created to check if the csv file of a category exists. If so, that category will be skipped.
7. To facilitate organizing csv file, a folder is created to contain all csv files of each genre.

## 8. Future Work

- One of the future work is convert the rating into numbers so that we could manipulate the data. For example, we can find the maximum or minimum rating, and find some statistic information.
- Moreover, we could extract the number of the reviews and relate to the rating star. For example, if the rating is 4.9/5 but there are few reviews, it will not be reliable.

## 9. Reference

**Refernce:**

https://jovian.ai/learn/zero-to-data-analyst-bootcamp/assignment/project-web-scraping-with-python